In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Cargar los datos
file_path = r'C:\Users\gcordero\Documents\Github\Data_Science_portfolio\Machine_Learning\Calendarizacion de Transportes\Consolidado de Transporte.txt'

# Leer el archivo con encoding latin-1 para manejar caracteres especiales
df = pd.read_csv(file_path, 
                 sep='\t', 
                 encoding='latin-1',
                 decimal=',',
                 thousands='.')

# Ver las primeras filas
print("Primeras filas del dataset:")
print(df.head())
print("\n" + "="*80 + "\n")

# Información general
print("Información del dataset:")
print(df.info())
print("\n" + "="*80 + "\n")

# Dimensiones
print(f"Dimensiones del dataset: {df.shape[0]} filas y {df.shape[1]} columnas")

Primeras filas del dataset:
  Transportista        Origen  Destino                  Ruta Fecha Expedicion  \
0           PDQ   Proveedores   Osorno  Proveedor - Sucursal       08-10-2025   
1           PDQ   Proveedores  Iquique  Proveedor - Sucursal       14-10-2025   
2           PDQ         Matta  Iquique   Santiago - Sucursal       14-10-2025   
3           PDQ   Proveedores  Iquique  Proveedor - Sucursal       14-10-2025   
4           PDQ  Puerto Montt    Matta   Sucursal - Sucursal       14-10-2025   

   Periodo Facturacion  Volumen   Peso   Importe   
0                45962    0.840  210.0   $33.600   
1                45962    0.041   12.0    $4.200   
2                45962    0.007    5.0    $4.200   
3                45962    0.139   35.0    $7.350   
4                45962    0.675  169.0   $14.365   


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 9 columns):
 #   Column               Non-Null Cou

In [5]:
# Limpiar nombres de columnas (eliminar espacios)
df.columns = df.columns.str.strip()

# Función para limpiar y convertir importes
def limpiar_importe(valor):
    if pd.isna(valor):
        return np.nan
    # Remover $, espacios y convertir
    valor_limpio = str(valor).replace('$', '').replace('.', '').replace(' ', '').strip()
    try:
        return float(valor_limpio)
    except:
        return np.nan

# Aplicar limpieza a la columna Importe
df['Importe'] = df['Importe'].apply(limpiar_importe)

# Convertir Fecha Expedicion a datetime (robusto)
# Guardar copia del valor original para depuración
df['Fecha_Expedicion_original'] = df['Fecha Expedicion']

# Intentar parsear con dayfirst=True (maneja 'DD-MM-YYYY' y formatos similares)
df['Fecha Expedicion'] = pd.to_datetime(
    df['Fecha Expedicion'],
    dayfirst=True,
    infer_datetime_format=True,
    errors='coerce'
)

# Informar si hubo valores no parseados y mostrar ejemplos
n_unparseadas = df['Fecha Expedicion'].isna().sum()
print(f"Fechas no parseadas: {n_unparseadas}")
if n_unparseadas > 0:
    print("Ejemplos de valores de fecha que no se pudieron convertir:")
    print(df.loc[df['Fecha Expedicion'].isna(), 'Fecha_Expedicion_original'].unique()[:20])
# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\n" + "="*80 + "\n")

# Estadísticas descriptivas
print("Estadísticas descriptivas de variables numéricas:")
print(df[['Volumen', 'Peso', 'Importe']].describe())
print("\n" + "="*80 + "\n")

# Verificar el rango de fechas
print(f"Rango de fechas: {df['Fecha Expedicion'].min()} a {df['Fecha Expedicion'].max()}")
print(f"Periodo de análisis: {(df['Fecha Expedicion'].max() - df['Fecha Expedicion'].min()).days} días")

Fechas no parseadas: 0
Valores nulos por columna:
Transportista                  0
Origen                         0
Destino                        0
Ruta                           0
Fecha Expedicion               0
Periodo Facturacion            0
Volumen                       77
Peso                         126
Importe                       31
Fecha_Expedicion_original      0
dtype: int64


Estadísticas descriptivas de variables numéricas:
            Volumen          Peso       Importe
count  12002.000000  11953.000000  1.204800e+04
mean       0.900787    264.003585  4.132760e+04
std        2.456273    695.204828  1.026490e+05
min        0.000000      0.000000  1.900000e+02
25%        0.031000     10.000000  5.104000e+03
50%        0.171428     52.000000  1.226200e+04
75%        0.864000    250.000000  3.668650e+04
max       72.000000  19250.000000  2.729294e+06


Rango de fechas: 2025-05-29 00:00:00 a 2025-11-20 00:00:00
Periodo de análisis: 175 días


In [8]:
# Análisis de Transportistas
print("=" * 80)
print("ANÁLISIS DE TRANSPORTISTAS")
print("=" * 80)
print("\nTransportistas únicos:")
print(df['Transportista'].value_counts())
print("\nTotal de transportistas diferentes:", df['Transportista'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE DESTINOS")
print("=" * 80)
print("\nTop 20 Destinos más frecuentes:")
print(df['Destino'].value_counts().head(20))
print("\nTotal de destinos diferentes:", df['Destino'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE ORÍGENES")
print("=" * 80)
print("\nOrígenes más frecuentes:")
print(df['Origen'].value_counts())
print("\nTotal de orígenes diferentes:", df['Origen'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE COSTOS")
print("=" * 80)
# Filtrar registros con importe válido
df_valid = df[df['Importe'] > 1].copy()
print(f"\nRegistros con importe válido: {len(df_valid)} de {len(df)}")
print(f"Costo total del periodo: ${df_valid['Importe'].sum():,.0f}")
print(f"Costo promedio por envío: ${df_valid['Importe'].mean():,.0f}")
print(f"Costo mediano por envío: ${df_valid['Importe'].median():,.0f}")

ANÁLISIS DE TRANSPORTISTAS

Transportistas únicos:
Transportista
PDQ      6248
SAMEX    3223
TVP      2608
Name: count, dtype: int64

Total de transportistas diferentes: 3

ANÁLISIS DE DESTINOS

Top 20 Destinos más frecuentes:
Destino
Puerto Montt        1142
Antofagasta         1004
Concepcion           983
Matta                869
Temuco               859
Valdivia             857
Iquique              804
Calama               698
Talca                690
Copiapo              647
La Serena            614
Los Angeles          538
Chillan              522
Rancagua             512
Osorno               443
Valparaiso           213
Curauma              194
Los Libertadores     165
CD Libertadores      120
Proveedores           90
Name: count, dtype: int64

Total de destinos diferentes: 23

ANÁLISIS DE ORÍGENES

Orígenes más frecuentes:
Origen
Proveedores         3131
Matta               1926
CD Lo Ruiz           760
CD Libertadores      614
Temuco               607
Antofagasta          586


In [10]:
# Crear columna de Ruta
df_valid['Ruta'] = df_valid['Ruta']

print("=" * 80)
print("TOP 20 RUTAS MÁS FRECUENTES")
print("=" * 80)
rutas_frecuencia = df_valid['Ruta'].value_counts().head(20)
print(rutas_frecuencia)

print("\n" + "=" * 80)
print("TOP 20 RUTAS MÁS COSTOSAS (COSTO TOTAL ACUMULADO)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(20))

print("\n" + "=" * 80)
print("TOP 20 RUTAS CON MAYOR COSTO PROMEDIO POR ENVÍO")
print("=" * 80)
# Filtrar rutas con al menos 10 envíos para que sea representativo
rutas_min_10 = rutas_costo_total[rutas_costo_total['count'] >= 10].sort_values('mean', ascending=False)
print(rutas_min_10.head(20))

print("\n" + "=" * 80)
print("RESUMEN DE ANÁLISIS DE RUTAS")
print("=" * 80)
print(f"Total de rutas únicas: {df_valid['Ruta'].nunique()}")
print(f"Rutas que representan el 80% del costo total: ", end='')
# Calcular Pareto (80/20)
costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(rutas_80)
print(f"Rutas que representan el 80% de la frecuencia: ", end='')
freq_acumulada = rutas_frecuencia.sort_values(ascending=False).cumsum()
total_freq = rutas_frecuencia.sum()
rutas_80_freq = (freq_acumulada <= total_freq * 0.8).sum()
print(rutas_80_freq)

TOP 20 RUTAS MÁS FRECUENTES
Ruta
Sucursal - Sucursal       4816
Santiago - Sucursal       3524
Proveedor - Sucursal      3127
Sucursal - Cliente         541
Santiago - Cliente          37
Proveedores - Clientes       3
Name: count, dtype: int64

TOP 20 RUTAS MÁS COSTOSAS (COSTO TOTAL ACUMULADO)
                                sum  count          mean
Ruta                                                    
Santiago - Sucursal     218140034.0   3524  61901.258229
Proveedor - Sucursal    127192217.0   3127  40675.477135
Sucursal - Sucursal     124703966.0   4816  25893.680648
Sucursal - Cliente       26125084.0    541  48290.358595
Santiago - Cliente        1645669.0     37  44477.540541
Proveedores - Clientes     107895.0      3  35965.000000

TOP 20 RUTAS CON MAYOR COSTO PROMEDIO POR ENVÍO
                              sum  count          mean
Ruta                                                  
Santiago - Sucursal   218140034.0   3524  61901.258229
Sucursal - Cliente     26125084.0 

In [18]:
# Normalizacion Columna de Origen
print("=" * 80)
print("NORMALIZACIÓN DE ORÍGENES")
print("=" * 80)

# Ver todos los orígenes únicos para identificar cuáles normalizar
print("\nTodos los orígenes únicos:")
origenes_unicos = sorted(df_valid['Origen'].unique())
for i, origen in enumerate(origenes_unicos, 1):
    print(f"{i}. {origen}")

print("\n" + "=" * 80)
print("APLICANDO NORMALIZACIÓN")
print("=" * 80)

# Función para normalizar orígenes
def normalizar_origen(origen):
    origen_upper = str(origen).upper()
    
    # Todo lo que contenga estos términos es SANTIAGO
    terminos_santiago = ['SANTIAGO', 'LAMPA', 'COLINA', 'QUILICURA', 'RENCA', 
                         'PUDAHUEL', 'VITACURA', 'MATTA', 'LOS LIBERTADORES',
                         'LO RUIZ', 'RUIZ', 'DARTEL', 'CD LIBERTADORES']
    
    for termino in terminos_santiago:
        if termino in origen_upper:
            return 'SANTIAGO'
    
    # Valparaíso
    if 'CURAUMA' in origen_upper or 'VALPARAISO' in origen_upper:
        return 'VALPARAISO'
    
    # Si no coincide con nada, mantener el original (normalizado a mayúsculas)
    return origen.upper()

# Aplicar normalización
df_valid['Origen_Normalizado'] = df_valid['Origen'].apply(normalizar_origen)

# Rehacer la columna Ruta con origen normalizado
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

print("\nOrígenes ANTES de normalizar:")
print(df['Origen'].value_counts().head(20))

print("\n" + "=" * 80)
print("Orígenes DESPUÉS de normalizar:")
print(df_valid['Origen_Normalizado'].value_counts())

print("\n" + "=" * 80)
print("NUEVAS TOP 20 RUTAS MÁS FRECUENTES (NORMALIZADAS)")
print("=" * 80)
rutas_frecuencia = df_valid['Ruta'].value_counts().head(20)
print(rutas_frecuencia)

print("\n" + "=" * 80)
print("NUEVAS TOP 20 RUTAS MÁS COSTOSAS (NORMALIZADAS)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(20))

print("\n" + "=" * 80)
print("RESUMEN ACTUALIZADO")
print("=" * 80)
print(f"Total de rutas únicas (normalizadas): {df_valid['Ruta'].nunique()}")
costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(f"Rutas que representan el 80% del costo total: {rutas_80}")

NORMALIZACIÓN DE ORÍGENES

Todos los orígenes únicos:
1. Antofagasta
2. CD Libertadores
3. CD Lo Ruiz
4. Calama
5. Chillan
6. Concepcion
7. Copiapo
8. Coquimbo
9. Curauma
10. Iquique
11. La Serena
12. Lira
13. Los Angeles
14. Los Libertadores
15. Matta
16. Osorno
17. Proveedores
18. Puerto Montt
19. Rancagua
20. Talca
21. Temuco
22. Valdivia
23. Valparaiso
24. Vitacura

APLICANDO NORMALIZACIÓN

Orígenes ANTES de normalizar:
Origen
Proveedores         3131
Matta               1926
CD Lo Ruiz           760
CD Libertadores      614
Temuco               607
Antofagasta          586
Puerto Montt         506
Concepcion           496
Talca                465
Valdivia             350
La Serena            327
Rancagua             296
Los Libertadores     274
Lira                 272
Calama               256
Copiapo              253
Chillan              250
Curauma              235
Osorno               219
Los Angeles          133
Name: count, dtype: int64

Orígenes DESPUÉS de normalizar:
Origen

In [19]:
# Agregar columnas de tiempo
df_valid['Mes'] = df_valid['Fecha Expedicion'].dt.to_period('M')
df_valid['Semana'] = df_valid['Fecha Expedicion'].dt.to_period('W')
df_valid['Dia_Semana'] = df_valid['Fecha Expedicion'].dt.day_name()

print("=" * 80)
print("ANÁLISIS DE FRECUENCIA TEMPORAL")
print("=" * 80)

print("\nEnvíos por mes:")
envios_mes = df_valid.groupby('Mes').agg({
    'Importe': ['sum', 'count', 'mean']
}).round(0)
envios_mes.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(envios_mes)
print(f"\nPromedio mensual: {envios_mes['Num Envíos'].mean():.0f} envíos, ${envios_mes['Costo Total'].mean():,.0f}")

print("\n" + "=" * 80)
print("FRECUENCIA POR DÍA DE LA SEMANA")
print("=" * 80)
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
envios_dia = df_valid.groupby('Dia_Semana').agg({
    'Importe': ['sum', 'count', 'mean']
}).reindex(dias_orden)
envios_dia.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(envios_dia)

print("\n" + "=" * 80)
print("ANÁLISIS DE FRECUENCIA PARA TOP 10 RUTAS MÁS COSTOSAS")
print("=" * 80)
top_10_rutas = rutas_costo_total.head(10).index
for ruta in top_10_rutas:
    df_ruta = df_valid[df_valid['Ruta'] == ruta]
    dias_entre_envios = df_ruta['Fecha Expedicion'].sort_values().diff().dt.days.dropna()
    
    print(f"\n{ruta}")
    print(f"  Total envíos: {len(df_ruta)}")
    print(f"  Costo total: ${df_ruta['Importe'].sum():,.0f}")
    print(f"  Frecuencia promedio: cada {dias_entre_envios.mean():.1f} días")
    print(f"  Frecuencia mediana: cada {dias_entre_envios.median():.1f} días")
    print(f"  Envíos/mes estimados: {30/dias_entre_envios.mean():.1f}")
    print(f"  Peso promedio: {df_ruta['Peso'].mean():.1f} kg")
    print(f"  Volumen promedio: {df_ruta['Volumen'].mean():.2f} m³")

ANÁLISIS DE FRECUENCIA TEMPORAL

Envíos por mes:
         Costo Total  Num Envíos  Costo Promedio
Mes                                             
2025-09  146253710.0        3760         38897.0
2025-10  179587302.0        4094         43866.0
2025-11   95187525.0        2473         38491.0

Promedio mensual: 3442 envíos, $140,342,846

FRECUENCIA POR DÍA DE LA SEMANA
            Costo Total  Num Envíos  Costo Promedio
Dia_Semana                                         
Monday       88859498.0      2140.0    41523.129907
Tuesday      97522568.0      2390.0    40804.421757
Wednesday    88007155.0      2081.0    42290.800096
Thursday     81917223.0      2094.0    39119.972779
Friday       63118740.0      1604.0    39350.835411
Saturday      1603353.0        18.0    89075.166667
Sunday              NaN         NaN             NaN

ANÁLISIS DE FRECUENCIA PARA TOP 10 RUTAS MÁS COSTOSAS

SANTIAGO → Antofagasta
  Total envíos: 244
  Costo total: $25,071,015
  Frecuencia promedio: cada 0.3 dí

In [20]:
print("=" * 80)
print("FILTRADO DE DATOS - PERIODO REAL DE ANÁLISIS")
print("=" * 80)

# Filtrar usando los objetos Period correctos
meses_principales = [pd.Period('2025-09', 'M'), pd.Period('2025-10', 'M'), pd.Period('2025-11', 'M')]
df_valid_periodo = df_valid[df_valid['Mes'].isin(meses_principales)].copy()

print(f"\nRegistros totales antes del filtro: {len(df_valid)}")
print(f"Registros en periodo real (Sep-Nov 2025): {len(df_valid_periodo)}")
print(f"Registros descartados (rezagos): {len(df_valid) - len(df_valid_periodo)}")

print("\n" + "=" * 80)
print("RESUMEN DEL PERIODO REAL (SEP-NOV 2025)")
print("=" * 80)

resumen_periodo = df_valid_periodo.groupby('Mes').agg({
    'Importe': ['sum', 'count', 'mean']
}).round(0)
resumen_periodo.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(resumen_periodo)
print(f"\n{'='*80}")
print(f"TOTALES DEL PERIODO (3 MESES):")
print(f"  Total envíos: {len(df_valid_periodo):,}")
print(f"  Costo total: ${df_valid_periodo['Importe'].sum():,.0f}")
print(f"  Promedio mensual: {len(df_valid_periodo)/3:.0f} envíos")
print(f"  Costo promedio mensual: ${df_valid_periodo['Importe'].sum()/3:,.0f}")
print(f"  Costo promedio por envío: ${df_valid_periodo['Importe'].mean():,.0f}")

# Actualizar dataset de trabajo
df_valid = df_valid_periodo.copy()

print("\n" + "=" * 80)
print("TOP 15 RUTAS MÁS COSTOSAS (PERIODO REAL)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(15))

costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(f"\nRutas que representan el 80% del costo: {rutas_80}")

FILTRADO DE DATOS - PERIODO REAL DE ANÁLISIS

Registros totales antes del filtro: 10327
Registros en periodo real (Sep-Nov 2025): 10327
Registros descartados (rezagos): 0

RESUMEN DEL PERIODO REAL (SEP-NOV 2025)
         Costo Total  Num Envíos  Costo Promedio
Mes                                             
2025-09  146253710.0        3760         38897.0
2025-10  179587302.0        4094         43866.0
2025-11   95187525.0        2473         38491.0

TOTALES DEL PERIODO (3 MESES):
  Total envíos: 10,327
  Costo total: $421,028,537
  Promedio mensual: 3442 envíos
  Costo promedio mensual: $140,342,846
  Costo promedio por envío: $40,770

TOP 15 RUTAS MÁS COSTOSAS (PERIODO REAL)
                                   sum  count           mean
Ruta                                                        
SANTIAGO → Antofagasta      25071015.0    244  102750.061475
SANTIAGO → Iquique          21065536.0    245   85981.779592
SANTIAGO → Puerto Montt     20556371.0    279   73678.749104
SANTIA

In [23]:
print("=" * 80)
print("ANÁLISIS DE COSTOS POR TRANSPORTISTA")
print("=" * 80)

# Análisis general por transportista
transportista_stats = df_valid.groupby('Transportista').agg({
    'Importe': ['sum', 'count', 'mean', 'median'],
    'Peso': 'sum',
    'Volumen': 'sum'
}).round(0)
transportista_stats.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio', 'Costo Mediano', 'Peso Total (kg)', 'Volumen Total (m³)']
print(transportista_stats)

print("\n" + "=" * 80)
print("COSTO POR KG Y M³ POR TRANSPORTISTA")
print("=" * 80)
transportista_stats['$/kg'] = (transportista_stats['Costo Total'] / transportista_stats['Peso Total (kg)']).round(0)
transportista_stats['$/m³'] = (transportista_stats['Costo Total'] / transportista_stats['Volumen Total (m³)']).round(0)
print(transportista_stats[['Num Envíos', 'Costo Promedio', '$/kg', '$/m³']])

print("\n" + "=" * 80)
print("TOP 5 RUTAS: COMPARACIÓN DE COSTOS POR TRANSPORTISTA")
print("=" * 80)

top_5_rutas = rutas_costo_total.head(5).index

for ruta in top_5_rutas:
    print(f"\n{ruta}:")
    df_ruta = df_valid[df_valid['Ruta'] == ruta]
    comparacion = df_ruta.groupby('Transportista').agg({
        'Importe': ['count', 'mean', 'median', 'sum'],
        'Peso': 'mean',
        'Volumen': 'mean'
    }).round(0)
    comparacion.columns = ['Envíos', 'Costo Promedio', 'Costo Mediano', 'Costo Total', 'Peso Prom (kg)', 'Vol Prom (m³)']
    if len(comparacion) > 0:
        print(comparacion.sort_values('Costo Promedio', ascending=False))

ANÁLISIS DE COSTOS POR TRANSPORTISTA
               Costo Total  Num Envíos  Costo Promedio  Costo Mediano  \
Transportista                                                           
PDQ            130397587.0        5365         24305.0         7416.0   
SAMEX          172961718.0        2703         63989.0        18092.0   
TVP            117669232.0        2259         52089.0        23571.0   

               Peso Total (kg)  Volumen Total (m³)  
Transportista                                       
PDQ                   778985.0              2543.0  
SAMEX                 998969.0              3996.0  
TVP                   885220.0              2561.0  

COSTO POR KG Y M³ POR TRANSPORTISTA
               Num Envíos  Costo Promedio   $/kg     $/m³
Transportista                                            
PDQ                  5365         24305.0  167.0  51277.0
SAMEX                2703         63989.0  173.0  43284.0
TVP                  2259         52089.0  133.0  45947.0

TOP 

In [28]:
print("=" * 80)
print("RECARGA COMPLETA DE DATOS CON CONVERSIÓN CORRECTA")
print("=" * 80)

# Leer el archivo SIN conversión automática de decimales
df = pd.read_csv(file_path, 
                 sep='\t', 
                 encoding='latin-1')

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Función CORREGIDA para limpiar importes (trabajar con el texto original)
def limpiar_importe_correcto(valor):
    if pd.isna(valor):
        return np.nan
    # Convertir a string y limpiar
    valor_str = str(valor).replace('$', '').replace(' ', '').strip()
    
    # Reemplazar punto por nada (es separador de miles en formato chileno)
    # Reemplazar coma por punto (es separador decimal)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    
    try:
        return float(valor_str)
    except:
        return np.nan

# Aplicar limpieza
df['Importe'] = df['Importe'].apply(limpiar_importe_correcto)
df['Volumen'] = pd.to_numeric(df['Volumen'].astype(str).str.replace(',', '.'), errors='coerce')
df['Peso'] = pd.to_numeric(df['Peso'], errors='coerce')

# Convertir fechas
df['Fecha Expedicion'] = pd.to_datetime(df['Fecha Expedicion'], format='%Y-%m-%d', errors='coerce')

print("\nPrimeras 20 filas con importes CORREGIDOS:")
print(df[['Transportista', 'Destino', 'Peso', 'Importe']].head(20))

print("\n" + "=" * 80)
print("Estadísticas de Importe CORREGIDO:")
print(df['Importe'].describe())

print("\n¿Estos valores se ven correctos ahora?")

RECARGA COMPLETA DE DATOS CON CONVERSIÓN CORRECTA

Primeras 20 filas con importes CORREGIDOS:
   Transportista           Destino   Peso  Importe
0            PDQ            Osorno  210.0  33600.0
1            PDQ           Iquique   12.0   4200.0
2            PDQ           Iquique    5.0   4200.0
3            PDQ           Iquique   35.0   7350.0
4            PDQ             Matta  169.0  14365.0
5            PDQ             Matta    8.0   6100.0
6            PDQ           Iquique   26.0   5460.0
7            PDQ  Los Libertadores   16.0   4300.0
8            PDQ        Concepcion   64.0  19456.0
9            PDQ       Antofagasta    2.0   3700.0
10           PDQ       Antofagasta   55.0  10175.0
11           PDQ       Antofagasta    5.0   3700.0
12           PDQ           Iquique  300.0  42900.0
13           PDQ            Calama   15.0   7905.0
14           PDQ       Antofagasta  180.0  45720.0
15           PDQ       Antofagasta   16.0   3700.0
16           PDQ   CD Libertadores    1

In [30]:
print("=" * 80)
print("PREPROCESAMIENTO COMPLETO CON DATOS CORRECTOS")
print("=" * 80)

# Normalizar orígenes
def normalizar_origen(origen):
    origen_upper = str(origen).upper()
    terminos_santiago = ['SANTIAGO', 'LAMPA', 'COLINA', 'QUILICURA', 'RENCA', 
                         'PUDAHUEL', 'VITACURA', 'MATTA', 'LOS LIBERTADORES',
                         'LO RUIZ', 'RUIZ', 'DARTEL']
    for termino in terminos_santiago:
        if termino in origen_upper:
            return 'SANTIAGO'
    if 'CURAUMA' in origen_upper or 'VALPARAISO' in origen_upper:
        return 'VALPARAISO'
    return origen.upper()

# Aplicar transformaciones
df['Origen_Normalizado'] = df['Origen'].apply(normalizar_origen)
df['Ruta'] = df['Origen_Normalizado'] + ' → ' + df['Destino']
df['Mes'] = df['Fecha Expedicion'].dt.to_period('M')

# Filtrar datos válidos (importes > 1 para eliminar errores)
df_valid = df[df['Importe'] > 1].copy()

# Filtrar solo el periodo real (Sep-Nov 2025)
meses_principales = [pd.Period('2025-09', 'M'), pd.Period('2025-10', 'M'), pd.Period('2025-11', 'M')]
df_valid = df_valid[df_valid['Mes'].isin(meses_principales)].copy()

print(f"\nRegistros válidos en periodo Sep-Nov 2025: {len(df_valid):,}")
print(f"Costo total 3 meses: ${df_valid['Importe'].sum():,.0f}")
print(f"Costo promedio por envío: ${df_valid['Importe'].mean():,.0f}")
print(f"Promedio mensual: {len(df_valid)/3:.0f} envíos, ${df_valid['Importe'].sum()/3:,.0f}")

print("\n" + "=" * 80)
print("RESUMEN POR TRANSPORTISTA")
print("=" * 80)
resumen_transp = df_valid.groupby('Transportista').agg({
    'Importe': ['sum', 'count', 'mean'],
    'Peso': 'sum',
    'Volumen': 'sum'
})
resumen_transp.columns = ['Costo Total', 'Envíos', 'Costo Prom', 'Peso Total (kg)', 'Vol Total (m³)']
resumen_transp['$/kg'] = (resumen_transp['Costo Total'] / resumen_transp['Peso Total (kg)']).round(0)
print(resumen_transp)

print("\n" + "=" * 80)
print("TOP 15 RUTAS MÁS COSTOSAS")
print("=" * 80)
rutas_costo = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo.head(15))

PREPROCESAMIENTO COMPLETO CON DATOS CORRECTOS

Registros válidos en periodo Sep-Nov 2025: 0
Costo total 3 meses: $0
Costo promedio por envío: $nan
Promedio mensual: 0 envíos, $0

RESUMEN POR TRANSPORTISTA
Empty DataFrame
Columns: [Costo Total, Envíos, Costo Prom, Peso Total (kg), Vol Total (m³), $/kg]
Index: []

TOP 15 RUTAS MÁS COSTOSAS
Empty DataFrame
Columns: [sum, count, mean]
Index: []


In [40]:
print("=" * 80)
print("NORMALIZACIÓN DE DESTINOS")
print("=" * 80)

print("\nDestinos únicos ANTES de normalizar:")
print(df_valid['Destino'].value_counts().head(30))

# Función para normalizar destinos
def normalizar_destino(destino):
    destino_upper = str(destino).upper().strip()
    
    # Normalizar Puerto Montt
    if 'MONTT' in destino_upper and 'PUERTO' not in destino_upper:
        return 'Puerto Montt'
    
    # Normalizar CD Libertadores
    if 'LIBERTADORES' in destino_upper or 'LOS LIBERTADORES' in destino_upper:
        return 'CD Libertadores'
    
    # Capitalizar correctamente destinos comunes
    normalizaciones = {
        'ANTOFAGASTA': 'Antofagasta',
        'IQUIQUE': 'Iquique',
        'CALAMA': 'Calama',
        'COPIAPO': 'Copiapo',
        'SERENA': 'Serena',
        'VALPARAISO': 'Valparaiso',
        'RANCAGUA': 'Rancagua',
        'TALCA': 'Talca',
        'CHILLAN': 'Chillan',
        'CONCEPCION': 'Concepcion',
        'TEMUCO': 'Temuco',
        'VALDIVIA': 'Valdivia',
        'OSORNO': 'Osorno',
        'PUERTO MONTT': 'Puerto Montt',
        'LOS ANGELES': 'Los Angeles',
        'CURAUMA': 'Curauma',
        'MATTA': 'Matta',
        'COQUIMBO': 'Coquimbo'
    }
    
    return normalizaciones.get(destino_upper, destino.title())

# Aplicar normalización
df_valid['Destino'] = df_valid['Destino'].apply(normalizar_destino)

# Recrear la columna Ruta con destino normalizado
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

print("\n" + "=" * 80)
print("Destinos únicos DESPUÉS de normalizar:")
print(df_valid['Destino'].value_counts().head(30))

print("\n" + "=" * 80)
print("TOP 15 RUTAS ACTUALIZADAS (con normalización)")
print("=" * 80)
rutas_costo = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo.head(15))

NORMALIZACIÓN DE DESTINOS

Destinos únicos ANTES de normalizar:
Destino
Antofagasta            859
Concepcion             844
Matta                  756
Valdivia               747
Temuco                 735
Iquique                664
Talca                  610
Calama                 589
Montt                  587
Copiapo                550
Serena                 518
Los Angeles            481
Chillan                442
Rancagua               416
Puerto Montt           397
Osorno                 385
Valparaiso             179
Curauma                158
Los Libertadores       125
CD Libertadores         85
Otro                    77
Vitacura                45
Lira                    39
CD Los Libertadores     28
CD Lo Ruiz               9
La Serena                2
Name: count, dtype: int64

Destinos únicos DESPUÉS de normalizar:
Destino
Puerto Montt       984
Antofagasta        859
Concepcion         844
Matta              756
Valdivia           747
Temuco             735
Iquique       

In [43]:
print("=" * 80)
print("CORRECCIÓN FINAL: NORMALIZACIÓN VALPARAÍSO Y RUTAS OPTIMIZADAS")
print("=" * 80)

# Normalizar Curauma como Valparaíso
df_valid.loc[df_valid['Destino'] == 'Curauma', 'Destino'] = 'Valparaiso'
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

# Recalcular desde Santiago
df_santiago = df_valid[df_valid['Origen_Normalizado'] == 'SANTIAGO'].copy()

# Calcular volúmenes SEMANALES
volumenes_semanales = df_santiago.groupby('Destino').agg({
    'Volumen': 'sum',
    'Peso': 'sum',
    'Importe': ['sum', 'count']
})
volumenes_semanales.columns = ['Vol Total', 'Peso Total', 'Costo Total', 'Envíos']
volumenes_semanales['Vol/semana (m³)'] = (volumenes_semanales['Vol Total'] / 12).round(1)
volumenes_semanales['Envíos/semana'] = (volumenes_semanales['Envíos'] / 12).round(1)
volumenes_semanales['% Ocupación'] = (volumenes_semanales['Vol/semana (m³)'] / 50 * 100).round(1)
volumenes_semanales['Costo/semana'] = (volumenes_semanales['Costo Total'] / 12).round(0)
volumenes_semanales = volumenes_semanales.sort_values('Vol/semana (m³)', ascending=False)

print("\nVOLÚMENES SEMANALES ACTUALIZADOS (Top 20):")
print("-" * 80)
print(f"{'Destino':<20} {'m³/sem':<10} {'%Ocup':<10} {'Envíos/sem':<12} {'Costo/sem'}")
print("-" * 80)
for dest, row in volumenes_semanales.head(20).iterrows():
    print(f"{dest:<20} {row['Vol/semana (m³)']:<10.1f} {row['% Ocupación']:<10.1f} {row['Envíos/semana']:<12.1f} ${row['Costo/semana']:,.0f}")

print("\n" + "=" * 80)
print("PROPUESTA FINAL DE RUTAS (BASADA EN TU FEEDBACK)")
print("=" * 80)

# Definir rutas finales
rutas_finales = [
    {
        'nombre': 'Ruta 1A: Santiago → Antofagasta (DIRECTO)',
        'destinos': ['Antofagasta'],
        'vol_estimado': 50.0,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Carga completa directa'
    },
    {
        'nombre': 'Ruta 1B: Santiago → Antofagasta → Calama (COMPARTIDO)',
        'destinos': ['Antofagasta', 'Calama'],
        'vol_estimado': None,  # Calcular
        'frecuencia': 'Semanal',
        'camiones': 2,
        'notas': 'Antofagasta residual (37 m³) + Calama (38 m³) = ~75 m³ = 2 camiones'
    },
    {
        'nombre': 'Ruta 2: Santiago → Iquique (DIRECTO)',
        'destinos': ['Iquique'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Carga completa'
    },
    {
        'nombre': 'Ruta 3: Santiago → Serena (DIRECTO)',
        'destinos': ['Serena'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~49 m³/semana - casi lleno'
    },
    {
        'nombre': 'Ruta 4: Santiago → Copiapo (DIRECTO)',
        'destinos': ['Copiapo'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~41 m³/semana - 82% ocupación'
    },
    {
        'nombre': 'Ruta 5: Santiago → Puerto Montt (DIRECTO)',
        'destinos': ['Puerto Montt'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~42 m³/semana - 84% ocupación'
    },
    {
        'nombre': 'Ruta 6: Santiago → Talca → Chillan',
        'destinos': ['Talca', 'Chillan'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Talca de paso hacia Chillán'
    },
    {
        'nombre': 'Ruta 7: Santiago → Los Angeles → Concepcion',
        'destinos': ['Los Angeles', 'Concepcion'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Ruta natural Sur'
    },
    {
        'nombre': 'Ruta 8: Santiago → Temuco → Valdivia',
        'destinos': ['Temuco', 'Valdivia'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Costa Sur consolidada'
    },
    {
        'nombre': 'Ruta 9: Santiago → Osorno → Puerto Montt (Alternativa)',
        'destinos': ['Osorno', 'Puerto Montt'],
        'vol_estimado': None,
        'frecuencia': 'Según necesidad',
        'camiones': 1,
        'notas': 'Si Puerto Montt no llena camión solo'
    },
    {
        'nombre': 'Ruta 10: Santiago → Rancagua + Valparaiso (LOCAL)',
        'destinos': ['Rancagua', 'Valparaiso'],
        'vol_estimado': None,
        'frecuencia': 'Semanal o Quincenal',
        'camiones': 1,
        'notas': 'Rutas cortas metropolitanas'
    }
]

# Calcular volúmenes reales para cada ruta
print("\nDETALLE DE RUTAS PROPUESTAS:")
print("=" * 80)

for i, ruta in enumerate(rutas_finales, 1):
    print(f"\n{ruta['nombre']}")
    print("-" * 80)
    
    vol_total = 0
    for dest in ruta['destinos']:
        if dest in volumenes_semanales.index:
            vol = volumenes_semanales.loc[dest, 'Vol/semana (m³)']
            envios = volumenes_semanales.loc[dest, 'Envíos/semana']
            costo = volumenes_semanales.loc[dest, 'Costo/semana']
            vol_total += vol
            print(f"  {dest:<20} {vol:>6.1f} m³/sem  |  {envios:>5.1f} envíos/sem  |  ${costo:>10,.0f}/sem")
    
    if len(ruta['destinos']) > 1:
        print(f"  {'TOTAL RUTA':<20} {vol_total:>6.1f} m³/sem  |  Ocupación: {vol_total/50*100:.1f}%")
    
    print(f"  Frecuencia: {ruta['frecuencia']}")
    print(f"  Camiones/despacho: {ruta['camiones']}")
    print(f"  Nota: {ruta['notas']}")

print("\n" + "=" * 80)
print("RESUMEN EJECUTIVO")
print("=" * 80)
total_camiones = sum([r['camiones'] for r in rutas_finales[:8]])  # Rutas principales
print(f"\nTotal de camiones en operación semanal: {total_camiones}")
print(f"Costo actual total semanal: ${df_santiago['Importe'].sum() / 12:,.0f}")

CORRECCIÓN FINAL: NORMALIZACIÓN VALPARAÍSO Y RUTAS OPTIMIZADAS

VOLÚMENES SEMANALES ACTUALIZADOS (Top 20):
--------------------------------------------------------------------------------
Destino              m³/sem     %Ocup      Envíos/sem   Costo/sem
--------------------------------------------------------------------------------
Antofagasta          87.2       174.4      50.7         $3,794,743
Iquique              60.0       120.0      40.0         $3,040,957
Serena               48.7       97.4       33.5         $1,573,717
Puerto Montt         42.2       84.4       43.8         $2,328,403
Copiapo              41.1       82.2       32.9         $1,651,712
Calama               37.6       75.2       34.8         $1,740,194
Talca                37.2       74.4       35.1         $1,310,868
Temuco               35.6       71.2       32.8         $1,821,033
Concepcion           34.4       68.8       39.6         $1,447,679
Valdivia             24.0       48.0       25.3         $1,316

In [31]:
print("=" * 80)
print("CALENDARIO SEMANAL DE DESPACHOS - RANCAGUA Y VALPARAÍSO SEPARADOS")
print("=" * 80)

# Calendario con Rancagua y Valparaíso como rutas independientes
calendario_despachos = {
    'Lunes': [
        {
            'ruta': 'Ruta 1B: Santiago → Antofagasta → Calama',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Antofagasta', 'Calama'],
            'vol_m3': 50.0,
            'vol_total': 75.0,
            'excedente': 25.0,
            'distancia_km': 1558,
            'costo_semanal': (3794743 / 2) + 1740194,
            'nota': '1 camión lleno + 25m³ excedente por courier. Entrega mar/mié'
        },
        {
            'ruta': 'Ruta 10A: Santiago → Rancagua (LOCAL)',
            'camiones': 1,
            'tipo_camion': 'Camión mediano (18m³)',
            'destinos': ['Rancagua'],
            'vol_m3': 18.3,
            'vol_total': 18.3,
            'excedente': 0,
            'distancia_km': 120,
            'costo_semanal': 537337,
            'nota': 'Camión mediano. Ruta local sur. Entrega lunes mismo día'
        },
        {
            'ruta': 'Ruta 10B: Santiago → Valparaiso (LOCAL)',
            'camiones': 1,
            'tipo_camion': 'Camión pequeño (5m³)',
            'destinos': ['Valparaiso'],
            'vol_m3': 4.2,
            'vol_total': 4.2,
            'excedente': 0,
            'distancia_km': 120,
            'costo_semanal': 127130,
            'nota': 'Camión pequeño. Ruta local oeste. Entrega lunes mismo día'
        }
    ],
    'Martes': [
        {
            'ruta': 'Ruta 5: Santiago → Osorno → Puerto Montt',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Osorno', 'Puerto Montt'],
            'vol_m3': 49.6,
            'vol_total': 49.6,
            'excedente': 0,
            'distancia_km': 1016,
            'costo_semanal': 476652 + 2328403,
            'nota': '1 camión (99% ocupación - ÓPTIMO). Osorno de paso. Entrega miércoles'
        },
        {
            'ruta': 'Ruta 6: Santiago → Talca → Chillan',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Talca', 'Chillan'],
            'vol_m3': 50.0,
            'vol_total': 59.1,
            'excedente': 9.1,
            'distancia_km': 407,
            'costo_semanal': 1310868 + 849308,
            'nota': '1 camión lleno + 9m³ excedente por courier. Entrega martes'
        }
    ],
    'Miércoles': [
        {
            'ruta': 'Ruta 8: Santiago → Temuco → Valdivia',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Temuco', 'Valdivia'],
            'vol_m3': 50.0,
            'vol_total': 59.6,
            'excedente': 9.6,
            'distancia_km': 840,
            'costo_semanal': 1821033 + 1316624,
            'nota': '1 camión lleno + 10m³ excedente por courier. Entrega jueves'
        },
        {
            'ruta': 'Ruta 7: Santiago → Los Angeles → Concepcion',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Los Angeles', 'Concepcion'],
            'vol_m3': 46.2,
            'vol_total': 46.2,
            'excedente': 0,
            'distancia_km': 515,
            'costo_semanal': 509660 + 1447679,
            'nota': '1 camión (92% ocupación). Entrega miércoles/jueves'
        }
    ],
    'Jueves': [
        {
            'ruta': 'Ruta 3: Santiago → Serena',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Serena'],
            'vol_m3': 48.7,
            'vol_total': 48.7,
            'excedente': 0,
            'distancia_km': 471,
            'costo_semanal': 1573717,
            'nota': '1 camión (97% ocupación). Entrega jueves/viernes'
        },
        {
            'ruta': 'Ruta 4: Santiago → Copiapo',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Copiapo'],
            'vol_m3': 41.1,
            'vol_total': 41.1,
            'excedente': 0,
            'distancia_km': 804,
            'costo_semanal': 1651712,
            'nota': '1 camión (82% ocupación). Entrega viernes'
        }
    ],
    'Viernes': [
        {
            'ruta': 'Ruta 2: Santiago → Iquique (VIAJE FIN DE SEMANA)',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Iquique'],
            'vol_m3': 50.0,
            'vol_total': 60.0,
            'excedente': 10.0,
            'distancia_km': 1849,
            'costo_semanal': 3040957,
            'nota': '🚚 Sale viernes, viaja FDS, entrega LUNES. 1 camión lleno + 10m³ courier'
        },
        {
            'ruta': 'Ruta 1A: Santiago → Antofagasta DIRECTO (VIAJE FIN DE SEMANA)',
            'camiones': 1,
            'tipo_camion': 'Camión grande (50m³)',
            'destinos': ['Antofagasta'],
            'vol_m3': 50.0,
            'vol_total': 87.2,
            'excedente': 37.2,
            'distancia_km': 1356,
            'costo_semanal': 3794743 / 2,
            'nota': '🚚 Sale viernes, viaja FDS, entrega LUNES. El resto va lunes compartido'
        }
    ]
}

# Mostrar calendario
print("\n📅 CALENDARIO SEMANAL DE DESPACHOS")
print("=" * 80)

total_camiones_semana = 0
total_costo_semana = 0
total_vol_flota = 0
total_excedente = 0
uso_rampas_por_dia = {}
flota_necesaria = {'grande': 0, 'mediano': 0, 'pequeño': 0}

dias_laborales = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

for dia, despachos in calendario_despachos.items():
    print(f"\n{'🟢 ' + dia.upper():=^80}")
    
    camiones_dia = sum([d['camiones'] for d in despachos if 'grande' in d['tipo_camion']])
    uso_rampas_por_dia[dia] = camiones_dia
    
    for despacho in despachos:
        total_camiones_semana += despacho['camiones']
        total_costo_semana += despacho['costo_semanal']
        total_vol_flota += despacho['vol_m3']
        total_excedente += despacho['excedente']
        
        # Contabilizar flota
        if 'grande' in despacho['tipo_camion']:
            flota_necesaria['grande'] += 1
        elif 'mediano' in despacho['tipo_camion']:
            flota_necesaria['mediano'] += 1
        elif 'pequeño' in despacho['tipo_camion']:
            flota_necesaria['pequeño'] += 1
        
        print(f"\n  {despacho['ruta']}")
        print(f"  {'─' * 76}")
        print(f"  Tipo: {despacho['tipo_camion']}")
        print(f"  Destinos: {' → '.join(despacho['destinos'])}")
        print(f"  Distancia: {despacho['distancia_km']} km")
        print(f"  Volumen: {despacho['vol_m3']:.1f} m³ de {despacho['vol_total']:.1f} m³ totales", end='')
        if despacho['excedente'] > 0:
            print(f"\n           ⚠️  Excedente: {despacho['excedente']:.1f} m³ por courier")
        else:
            print()
        print(f"  Costo actual/semana: ${despacho['costo_semanal']:,.0f}")
        print(f"  📝 {despacho['nota']}")

print("\n" + "=" * 80)
print("RESUMEN SEMANAL")
print("=" * 80)
print(f"\nTotal de despachos/semana: {total_camiones_semana}")
print(f"  - Camiones grandes (50m³): {flota_necesaria['grande']} despachos")
print(f"  - Camiones medianos (~18m³): {flota_necesaria['mediano']} despachos")
print(f"  - Camiones pequeños (~5m³): {flota_necesaria['pequeño']} despachos")
print(f"\nVolumen transportado flota propia: {total_vol_flota:.1f} m³/semana")
print(f"Volumen total generado: {total_vol_flota + total_excedente:.1f} m³/semana")
print(f"Excedente por courier: {total_excedente:.1f} m³/semana ({total_excedente/(total_vol_flota + total_excedente)*100:.1f}%)")
print(f"\nCosto actual total/semana: ${total_costo_semana:,.0f}")
print(f"Costo actual mensual (4 semanas): ${total_costo_semana * 4:,.0f}")
print(f"Costo actual anual proyectado: ${total_costo_semana * 52:,.0f}")

print("\n" + "=" * 80)
print("USO DE RAMPAS PRINCIPALES (CAMIONES GRANDES 50m³)")
print("=" * 80)
print(f"\n{'Día':<15} {'Camiones Grandes':<18} {'Estado':<40}")
print("-" * 80)
for dia in dias_laborales:
    camiones = uso_rampas_por_dia.get(dia, 0)
    if camiones == 2:
        indicador = "🟢🟢 COMPLETO"
    elif camiones == 1:
        indicador = "🟢⚪ PARCIAL"
    else:
        indicador = "⚪⚪ LIBRE"
    print(f"{dia:<15} {camiones:<18} {indicador}")

print("\n" + "=" * 80)
print("VISUALIZACIÓN SEMANAL")
print("=" * 80)
print("""
        LUNES                MARTES      MIÉRCOLES   JUEVES      VIERNES
    ┌────────────────┐   ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐
R1  │ Antofa.+Calama │   │ Osorno + │ │  Temuco  │ │  Serena  │ │ Iquique  │
    │   (50m³ G*)    │   │P. Montt  │ │    +     │ │          │ │ (FDS)    │
    │                │   │  50m³ G  │ │ Valdivia │ │  49m³ G  │ │  50m³ G  │
    │                │   │ (99%)    │ │  50m³ G  │ │          │ │          │
    ├────────────────┤   ├──────────┤ ├──────────┤ ├──────────┤ ├──────────┤
R2  │ Rancagua (18M*)│   │  Talca   │ │Los Ang.+ │ │ Copiapo  │ │Antofagas.│
    │                │   │    +     │ │Concepción│ │          │ │ DIRECTO  │
    │ Valparaíso(5P*)│   │ Chillan  │ │  46m³ G  │ │  41m³ G  │ │ (FDS)    │
    │                │   │  50m³ G  │ │          │ │          │ │  50m³ G  │
    └────────────────┘   └──────────┘ └──────────┘ └──────────┘ └──────────┘
      1 rampa grande     2 rampas     2 rampas    2 rampas     2 rampas
      +2 camiones local  grandes      grandes     grandes      grandes

    G = Camión Grande (50m³), M = Camión Mediano (18m³), P = Camión Pequeño (5m³)
    FDS = Viaja Fin De Semana, entrega Lunes
""")

print("\n💡 NOTAS OPERACIONALES:")
print("  ✓ Camiones grandes: 10 despachos/semana (usan rampas de 50m³)")
print("  ✓ Camiones locales lunes: Rancagua (mediano 18m³) + Valparaíso (pequeño 5m³)")
print("  ✓ Rancagua y Valparaíso separados (direcciones opuestas)")
print("  ✓ Ocupación promedio camiones grandes: {:.0f}%".format(total_vol_flota/500*100))
print("  ✓ Excedente: {:.1f} m³/semana ({:.1f}% del total)".format(total_excedente, total_excedente/(total_vol_flota + total_excedente)*100))
print("  ✓ VIERNES: Rutas extremas viajan fin de semana, entregan LUNES")

CALENDARIO SEMANAL DE DESPACHOS - RANCAGUA Y VALPARAÍSO SEPARADOS

📅 CALENDARIO SEMANAL DE DESPACHOS

====================================🟢 LUNES=====================================

  Ruta 1B: Santiago → Antofagasta → Calama
  ────────────────────────────────────────────────────────────────────────────
  Tipo: Camión grande (50m³)
  Destinos: Antofagasta → Calama
  Distancia: 1558 km
  Volumen: 50.0 m³ de 75.0 m³ totales
           ⚠️  Excedente: 25.0 m³ por courier
  Costo actual/semana: $3,637,566
  📝 1 camión lleno + 25m³ excedente por courier. Entrega mar/mié

  Ruta 10A: Santiago → Rancagua (LOCAL)
  ────────────────────────────────────────────────────────────────────────────
  Tipo: Camión mediano (18m³)
  Destinos: Rancagua
  Distancia: 120 km
  Volumen: 18.3 m³ de 18.3 m³ totales
  Costo actual/semana: $537,337
  📝 Camión mediano. Ruta local sur. Entrega lunes mismo día

  Ruta 10B: Santiago → Valparaiso (LOCAL)
  ──────────────────────────────────────────────────────────────